In [1]:
import pandas as pd
from modelling_prep import *
import os

/Users/katyscott/opt/miniconda3/envs/radiogenomics/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/71/rrp6j9vj3tv1mc89c__4w9bc0000gn/T/ipykernel_29085/3130051639.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
hnscc_clinical = pd.read_excel("/Users/katyscott/Documents/HNC Project/data/HNSCC/clinical_data/HNSCC-MDA-Data_update_20240514.xlsx")

In [ ]:
hnscc_clinical[hnscc_clinical['Follow up duration (month)'] > hnscc_clinical['Survival  (months)']]

## Add compactness to main radiomics file

In [2]:
datasetName = "HNSCC"

In [6]:
radFileName = "radiomicfeatures_" + datasetName + ".csv"
compactnessRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features_compactness/", radFileName))

baseRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features/", radFileName))

print(len(baseRadiomics))
print(len(compactnessRadiomics))

396
396


In [ ]:
negControls = ['shuffled_full', 'shuffled_roi', 'shuffled_non_roi','randomized_full', 'randomized_roi', 'randomized_non_roi', 'randomized_sampled_full',] # 'randomized_sampled_roi', 'randomized_sampled_non_roi']

for negControl in negControls:
    radFileName =  "radiomicfeatures_" + negControl + "_" + datasetName + ".csv"

    compactnessRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features_compactness/", radFileName))

    baseRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features/", radFileName))
    print("# of patients in base", negControl, "radiomics:", len(baseRadiomics))
    print("# of patients in compactness", negControl, "radiomics:", len(compactnessRadiomics))

In [9]:
negControl = "shuffled_non_roi"
radFileName =  "radiomicfeatures_" + negControl + "_" + datasetName + ".csv"

compactnessRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features_compactness/", radFileName))

baseRadiomics = pd.read_csv(os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "radiomic_features/", radFileName))

In [10]:
comRadID = getPatientIdentifierLabel(compactnessRadiomics)
baseRadID = getPatientIdentifierLabel(baseRadiomics)

compactnessRadiomics.set_index(comRadID, inplace=True)
baseRadiomics.set_index(baseRadID, inplace=True)

# Check that patient IDs match
if all(compactnessRadiomics.index == baseRadiomics.index):

    compactOnly = compactnessRadiomics['original_shape_Compactness1']

    originalRadiomicsCols = baseRadiomics.filter(regex=r'^original_*')
    insertIndex = baseRadiomics.columns.get_loc(originalRadiomicsCols.columns[0])

    baseRadiomics.insert(insertIndex, 'original_shape_Compactness1', compactOnly)

else:
    print("Mismatched patient IDs in compactness and base radiomics")

Multiple patient identifier labels found. Using the first one.
Multiple patient identifier labels found. Using the first one.


In [11]:
outputPath = os.path.join("/Users/katyscott/Documents/HNC Project/data/", datasetName, "total_radiomic_features/", radFileName)
baseRadiomics.to_csv(outputPath)

In [ ]:
for idx in range(0, len(compactnessRadiomics)):
    if not (baseRadiomics.iat[idx,0] == compactnessRadiomics.iat[idx,0]):
        print(idx)
        print("Base rad value:", baseRadiomics.iat[idx,0])
        print("Compact rad value:", compactnessRadiomics.iat[idx,0])
        break